In [ ]:
# %pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install transformers[torch]==4.30.2
# %pip install accelerate -U
# %pip install optuna
%pip install fastai spacy
%python -m spacy download en_core_web_sm


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Combined 2066 rows row datasets :

In [2]:
import pandas as pd

file_path =  "/content/drive/MyDrive/DistilBERT-7-Mei/combined_dataset.csv"
df = pd.read_csv(file_path)

In [3]:
print(df.shape)

(2066, 5)


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm

# 1. Load your dataset
# df = pd.read_csv("/content/combined_dataset.csv")
texts = df["clean_text"].tolist()
labels = df["encoded_label"].tolist()

# 2. Train-test split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# 3. Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
bert_model = AutoModel.from_pretrained("distilbert-base-uncased").eval().cuda()

# 4. Custom dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(X_train_texts, y_train, tokenizer)
test_dataset = TextDataset(X_test_texts, y_test, tokenizer)

# 5. Extract CLS embeddings
def extract_cls_embeddings(model, dataset, batch_size=16):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeds = outputs.last_hidden_state[:, 0, :]  # [CLS] token
            embeddings.append(cls_embeds.cpu().numpy())
    return np.vstack(embeddings)

X_train = extract_cls_embeddings(bert_model, train_dataset)
X_test = extract_cls_embeddings(bert_model, test_dataset)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Extracting embeddings: 100%|██████████| 26/26 [00:01<00:00, 18.74it/s]


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 6. Compute Pearson kernel
def pearson_kernel(X1, X2):
    X1_centered = X1 - X1.mean(axis=1, keepdims=True)
    X2_centered = X2 - X2.mean(axis=1, keepdims=True)
    num = np.dot(X1_centered, X2_centered.T)
    denom = np.linalg.norm(X1_centered, axis=1, keepdims=True) * np.linalg.norm(X2_centered, axis=1, keepdims=True).T
    return num / (denom + 1e-8)

K_train = pearson_kernel(X_train, X_train)
K_test = pearson_kernel(X_test, X_train)  # note: test vs train


In [6]:
# 7. Train SVM
svm = SVC(kernel="precomputed")
svm.fit(K_train, y_train)

from sklearn.metrics import classification_report

# 8. Predict and evaluate
y_pred = svm.predict(K_test)
f1 = f1_score(y_test, y_pred, average="macro")
print(f"\n🎯 Pearson-SVM F1 Score (macro): {f1:.4f}")

print("\n📋 Classification Report:\n")
print(classification_report(y_test, y_pred, digits=4))



🎯 Pearson-SVM F1 Score (macro): 0.7354

📋 Classification Report:

              precision    recall  f1-score   support

         0.0     0.7430    0.8114    0.7757       228
         1.0     0.7394    0.6559    0.6952       186

    accuracy                         0.7415       414
   macro avg     0.7412    0.7337    0.7354       414
weighted avg     0.7414    0.7415    0.7395       414



In [1]:
!git clone https://github.com/WilliamAxelC/Indonesian-Cyberbullying-Detection-with-Distilbert.git


Cloning into 'Indonesian-Cyberbullying-Detection-with-Distilbert'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 132 (delta 63), reused 111 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (132/132), 15.42 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil, os

# Nama file notebook sesuai dengan yang kamu save di Drive
notebook_name = "Added_Dataset_SVM_Distilbert_tokenizer.ipynb"
src_path = f"/content/drive/MyDrive/Colab Notebooks/{notebook_name}"
repo_dir = "/content/Indonesian-Cyberbullying-Detection-with-Distilbert"

if os.path.exists(src_path):
    shutil.copy(src_path, f"{repo_dir}/{notebook_name}")
    print(f"✅ Copied {notebook_name} ke repo folder.")
else:
    print(f"⚠️ Notebook tidak ditemukan di {src_path}. Periksa nama file!")
